In [3]:
from pathlib import Path


with open("all-output.txt", 'r') as f:
    lines = f.readlines()

lines = [l.split() for l in lines]
lines = [l for l in lines if len(l)>=7 and len(l[1])>=12]
pairs = dict([(l[1], int(l[2])) for l in lines])

rRNA_tRNA =set(["RF00001",
"RF00002",
"RF02543",
"RF02542",
"RF02541",
"RF02540",
"RF01960",
"RF00005",
"RF00177",
"RF01959",

# tRNA
"RF02541",
"RF02543",
"RF01852",
"RF00005",
"RF00233"])

rfam = Path("Rfam.pdb")
with open(rfam, "r") as f:
    rfam_lines = f.readlines()
rfam_lines = rfam_lines[1:]  # Skip the header line
rfam_lines = [line.split('\t')[:2] for line in rfam_lines]
# make dict mapping, upper(rfam_line[1]) to rfam_line[0]
rfam_dict = {line[1].upper(): line[0] for line in rfam_lines}
rfam_dict

{'1VQL': 'RF02540',
 '3J8G': 'RF02541',
 '4V64': 'RF02541',
 '5EL5': 'RF02541',
 '6FTG': 'RF02543',
 '6RI5': 'RF02543',
 '6WU9': 'RF02541',
 '6XU8': 'RF02543',
 '6ZMO': 'RF02543',
 '8BHF': 'RF02543',
 '8EKB': 'RF02541',
 '8FL6': 'RF02543',
 '8FZJ': 'RF02541',
 '1VY5': 'RF02541',
 '4LT8': 'RF02541',
 '4V73': 'RF02541',
 '4V7R': 'RF02543',
 '5IB8': 'RF02541',
 '6I0Y': 'RF02541',
 '6S0X': 'RF02541',
 '6SPB': 'RF02541',
 '6UZ7': 'RF02543',
 '6WD0': 'RF02541',
 '6YLH': 'RF02543',
 '7M4W': 'RF02541',
 '7TOR': 'RF02543',
 '7Z20': 'RF02541',
 '8CGV': 'RF02541',
 '8E47': 'RF02541',
 '8FLE': 'RF02543',
 '8Y0U': 'RF02543',
 '4U3M': 'RF02543',
 '4V5G': 'RF02541',
 '6ND6': 'RF02541',
 '6WDA': 'RF02541',
 '6WDB': 'RF02541',
 '7PH9': 'RF02541',
 '7QCA': 'RF02543',
 '8AKN': 'RF02541',
 '8YOO': 'RF02543',
 '4V7B': 'RF02541',
 '4V7L': 'RF02541',
 '4V7V': 'RF02541',
 '4V8I': 'RF02541',
 '5ADY': 'RF02541',
 '5HCP': 'RF02541',
 '5NJT': 'RF02541',
 '6OSI': 'RF02541',
 '6QT0': 'RF02543',
 '6XDQ': 'RF02541',


In [62]:
import pandas as pd
seg=1
data_dir = '../boltz_1seg.csv'
df = pd.read_csv(data_dir)
df

,pdb,rms,ermsd,inf,tmscore,gdts,lddt
0,1RLG_1_D_D_11_A_AA.pdb,0.288,0.816,1.000,0.0863,0.3864,0.962
1,1KH6_1_A_A_32_A_AA.pdb,0.334,0.265,1.000,0.0000,0.0000,1.000
2,5BTM_1_A_A_29_A_AA.pdb,0.351,0.733,1.000,0.0000,0.0000,0.796
3,5D5L_1_A_A_11_A_AA.pdb,0.365,0.293,1.000,0.0404,0.4091,0.979
4,5B2T_1_A_A_36_A_AA.pdb,0.373,0.839,1.000,0.0000,0.0000,0.975
...,...,...,...,...,...,...,...
1415,6YWS_1_A_A_838_G_AA.pdb,15.381,1.689,0.204,0.0000,0.0000,0.393
1416,4AOB_1_A_A_79_G_AA.pdb,16.043,1.833,0.125,0.0000,0.0000,0.451
1417,4M4O_1_B_B_38_U_AA.pdb,16.675,2.231,0.250,0.1297,0.2581,0.347
1418,4RMO_1_B_B_11_C_AA.pdb,16.915,2.375,0.684,0.1307,0.3065,0.370


In [63]:
df['pdb_id'] = [x.split('_')[0] for x in df['pdb']]
df['rfam_id'] = df['pdb_id'].apply(lambda x: rfam_dict.get(x.upper(), None))
# drop rows if rfam_id in rRNA_tRNA
df = df[~df['rfam_id'].isin(rRNA_tRNA)]

# add column base name. Take pdb column and do this: name = basename.split("_AA.pdb")[0]
df['base_name'] = df['pdb'].apply(lambda x: x.split("_AA.pdb")[0])
# The using base_name column, assign pairs values
df['pairs'] = df['base_name'].apply(lambda x: pairs.get(x, 0))
# finally, choose only records with pairs == seg
df = df[df['pairs'] == seg]
df

/tmp/ipykernel_868786/2867748707.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['base_name'] = df['pdb'].apply(lambda x: x.split("_AA.pdb")[0])
/tmp/ipykernel_868786/2867748707.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pairs'] = df['base_name'].apply(lambda x: pairs.get(x, 0))


,pdb,rms,ermsd,inf,tmscore,gdts,lddt,pdb_id,rfam_id,base_name,pairs
0,1RLG_1_D_D_11_A_AA.pdb,0.288,0.816,1.000,0.0863,0.3864,0.962,1RLG,None,1RLG_1_D_D_11_A,1
1,1KH6_1_A_A_32_A_AA.pdb,0.334,0.265,1.000,0.0000,0.0000,1.000,1KH6,None,1KH6_1_A_A_32_A,1
2,5BTM_1_A_A_29_A_AA.pdb,0.351,0.733,1.000,0.0000,0.0000,0.796,5BTM,None,5BTM_1_A_A_29_A,1
3,5D5L_1_A_A_11_A_AA.pdb,0.365,0.293,1.000,0.0404,0.4091,0.979,5D5L,None,5D5L_1_A_A_11_A,1
4,5B2T_1_A_A_36_A_AA.pdb,0.373,0.839,1.000,0.0000,0.0000,0.975,5B2T,None,5B2T_1_A_A_36_A,1
...,...,...,...,...,...,...,...,...,...,...,...
1414,4XWF_1_A_A_13_G_AA.pdb,15.364,1.076,1.000,0.1462,0.2981,0.500,4XWF,RF01750,4XWF_1_A_A_13_G,1
1416,4AOB_1_A_A_79_G_AA.pdb,16.043,1.833,0.125,0.0000,0.0000,0.451,4AOB,RF00162,4AOB_1_A_A_79_G,1
1417,4M4O_1_B_B_38_U_AA.pdb,16.675,2.231,0.250,0.1297,0.2581,0.347,4M4O,None,4M4O_1_B_B_38_U,1
1418,4RMO_1_B_B_11_C_AA.pdb,16.915,2.375,0.684,0.1307,0.3065,0.370,4RMO,None,4RMO_1_B_B_11_C,1


In [64]:
mean_rmsd = df['rms'].mean()
std_rmsd = df['rms'].std()
ermsd = df['ermsd'].mean()
std_ermsd = df['ermsd'].std()
mean_inf = df['inf'].mean()
std_inf = df['inf'].std()
mean_tmscore = df['tmscore'].mean()
std_tmscore = df['tmscore'].std()
mean_gdts = df['gdts'].mean()
std_gdts = df['gdts'].std()
mean_lddt = df['lddt'].mean()
std_lddt = df['lddt'].std()

# print in latex table format
print("mean rmsd, std rmsd, ermsd, std ermsd, mean inf, std inf, mean tmscore, std tmscore, mean gdts, std gdts, mean lddt, std lddt")
print(f"{mean_rmsd:.2f} $\\pm$ {std_rmsd:.2f} & {ermsd:.2f} $\\pm$ {std_ermsd:.2f} & {mean_inf:.2f} $\\pm$ {std_inf:.2f} & {mean_tmscore:.2f} $\\pm$ {std_tmscore:.2f} & {mean_gdts:.2f} $\\pm$ {std_gdts:.2f} & {mean_lddt:.2f} $\\pm$ {std_lddt:.2f} \\\\")

mean rmsd, std rmsd, ermsd, std ermsd, mean inf, std inf, mean tmscore, std tmscore, mean gdts, std gdts, mean lddt, std lddt
3.28 $\pm$ 3.17 & 1.25 $\pm$ 0.47 & 0.78 $\pm$ 0.33 & 0.07 $\pm$ 0.09 & 0.19 $\pm$ 0.24 & 0.71 $\pm$ 0.15 \\


In [47]:
2655+3752+1245

7652

In [66]:
10583-7652+33518

36449